In [ ]:
!python --version

Python 3.10.12


In [ ]:
import pandas as pd

#untuk dapat melihat seluruh kolom dan baris
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# Connect google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read TransJakarta Transaction Data

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/MSIB 6 (GreatEdu)/dfTransjakarta.csv")

In [ ]:
df.shape

(37900, 22)

In [ ]:
df.head()

# cara mensorting data

In [ ]:
# Menampilkan 5 nama penumpang dengan jenis kelamin Female(perempuan) dengan umur yang termuda diantara penumpang perempuan lainnya
# sorting by descending
Data_Female = df[df['payCardSex'] == 'F'].sort_values(by='payCardBirthDate', ascending=False).head(5)[['payCardName','payCardSex','payCardBirthDate']]
Data_Female

,payCardName,payCardSex,payCardBirthDate
4106,"R. Lili Permata, S.E.",F,2012
20322,Catur Najmudin,F,2012
36527,Rahmi Saptono,F,2012
10122,Puspa Sihombing,F,2012
14649,Sutan Nasim Saragih,F,2012


# membuat kolom baru 'umur' tahun 2024 dan jumlah transaksi yang dilakukan oleh penumpang pensiun (>55 tahun) lalu tuliskan juga jumlah masing-masing
transaksi perempuan dan laki-laki.

In [ ]:
df_add = df.copy()
df_add.head()

In [ ]:
import pandas as pd

from datetime import datetime
# menambahkan satuan kolom umur
tahun_2024 = datetime.now().year
df["Age"] = tahun_2024 - df["payCardBirthDate"]

# menghitung jumlah transaksi penumpang
jumlah_pensiun = (df["Age"] > 55). sum()
jumlah_transaksi_laki_laki = df[(df["payCardSex"] == "M")& (df["Age"] > 55)].shape[0]
jumlah_transaksi_perempuan = df[(df["payCardSex"] == "F")& (df["Age"] > 55)].shape[0]

# cetak
print(df["Age"])
print("="*100,"\n",
      "\nJumlah transaksi oleh penumpang yang sudah pensiun (Age > 55 tahun):", jumlah_pensiun,
      "\nJumlah transaksi penumpang laki-laki:", jumlah_transaksi_laki_laki,
      "\nJumlah transaksi penumpang perempuan:", jumlah_transaksi_perempuan)

0        16
1        27
2        32
3        46
4        42
         ..
37895    20
37896    50
37897    48
37898    48
37899    39
Name: Age, Length: 37900, dtype: int64
 
Jumlah transaksi oleh penumpang yang sudah pensiun (Age > 55 tahun): 2438 
Jumlah transaksi penumpang laki-laki: 1783 
Jumlah transaksi penumpang perempuan: 655


# Menampilkan data transaksi pertama dengan berpacu pada tapInTime menggunalan fungsi iloc

In [ ]:

detail = df.sort_values(by = "tapInTime").iloc[0]
print(detail)

transID                                  LDEA875J4U32YZ
payCardID                               213107623841273
payCardBank                                      emoney
payCardName                             Natalia Pratama
payCardSex                                            M
payCardBirthDate                                   1964
corridorID                                          11D
corridorName        Pulo Gebang - Pulo Gadung 2 via PIK
direction                                           0.0
tapInStops                                      B05823P
tapInStopsName                        United Tractors 1
tapInStopsLat                                  -6.18326
tapInStopsLon                                 106.93243
stopStartSeq                                         28
tapInTime                           2023-04-01 06:22:27
tapOutStops                                     B03090P
tapOutStopsName                       Raya Penggilingan
tapOutStopsLat                                -6

In [ ]:
# temukan transaksi gratis (0 rupiah)
transaksi_gratis= (df['payAmount'] == 0).sum()

# temukan penumpang yang melakukan transaksi perjalanan dengan jumlah pemberhentian terbanyak
df['Banyaknya_pemberhentian'] = df['stopEndSeq'] - df['stopStartSeq']
sortir_pemberhentian = df.sort_values(by='Banyaknya_pemberhentian', ascending=False)
transaksi_pemberhentian_terbanyak = sortir_pemberhentian.iloc[0]

# cetak hasil
print("Jumlah banyaknya transaksi gratis =",transaksi_gratis)
print("Informasi tentang penumpang dengan transaksi paling banyak pemberhentian:")
print("Nama Penumpang:", transaksi_pemberhentian_terbanyak["payCardName"])
print("Rute Perjalanan:", [transaksi_pemberhentian_terbanyak["tapInStopsName"],transaksi_pemberhentian_terbanyak["tapOutStopsName"]])
print("Total Pemberhentian:",transaksi_pemberhentian_terbanyak["Banyaknya_pemberhentian"])

Jumlah banyaknya transaksi gratis = 16648
Informasi tentang penumpang dengan transaksi paling banyak pemberhentian:
Nama Penumpang: Karen Pertiwi
Rute Perjalanan: ['RS Pelabuhan Jakarta', 'Kel. Rorotan']
Total Pemberhentian: 64.0


# menampilkan biaya yang dikeluarkan dan rute perjalanan untuk transaksi dengan jarak, tempuh terlama dan jumlah pemberhentian terbanyak

In [ ]:
# temukan transaksi dengan jarak tempuh terlama dengan menggunakan kolom tapInTime dan tapOutTime untuk mengetahui waktu terlama
df["tapInTime"] = pd.to_datetime(df["tapInTime"])
df["tapOutTime"] = pd.to_datetime(df["tapOutTime"])
df["durasi"] = (df["tapOutTime"] - df["tapInTime"]).dt.total_seconds()

# hitung jumlah pemberhentian
df["jumlah_pemberhentian"] = df["stopEndSeq"] - df["stopStartSeq"]

# mensortir berdasarkan kolom durasi(jarak tempuh terlama) dan jumlah pemberhentian
df = df.sort_values (by=["durasi", "jumlah_pemberhentian"], ascending = [False, False]).reset_index(drop=True)
jumlah_pemberhentian = df.loc[0, "jumlah_pemberhentian"]
rute_terlama = [df.loc[0, "tapInStopsName"], df.loc[0, "tapOutStopsName"]]
biaya = df.loc[0, "payAmount"]

# cetak hasil
print("Biaya perjalanan untuk transaksi dengan jarak tempuh terlama yang dikeluarkan sebanyak:", biaya, "rupiah")
print("Rute perjalanan untuk transaksi dengan jarak tempuh terlama:", rute_terlama)
print("Rute perjalanan untuk transaksi dengan jumlah pemberhentian terbanyak :", jumlah_pemberhentian, "kali pemberhentian")

Biaya perjalanan untuk transaksi dengan jarak tempuh terlama yang dikeluarkan sebanyak: 3500.0 rupiah
Rute perjalanan untuk transaksi dengan jarak tempuh terlama: ['Sbr. Jln. H. Ung', 'Jembatan Item']
Rute perjalanan untuk transaksi dengan jumlah pemberhentian terbanyak : 12.0 kali pemberhentian
